<a href="https://colab.research.google.com/github/saarang2003/Machine-Tranlator/blob/main/Machine_Language_Translator_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
import pandas as pd

# Load your CSV data (assuming your file is 'sentences.csv')
df = pd.read_csv('eng-hin.csv', header=None)

# Ensure rows with missing '-' symbol are handled gracefully
# Split the sentence on the '-' symbol, but only if it exists
df[['English', 'Hindi']] = df[0].str.split('-', expand=True, n=1)

# Drop rows where either English or Hindi is missing
df.dropna(subset=['English', 'Hindi'], inplace=True)

# Optionally, reset the index
df.reset_index(drop=True, inplace=True)

# Save the cleaned data to a new CSV file
df.to_csv('cleaned_sentences.csv', index=False)

# Display the cleaned dataframe
print(df)


                                                     0  \
0    I am learning to drive. - मैं गाड़ी चलाना सीख ...   
1    Can you recommend a good restaurant? - क्या आप...   
2    I need to buy groceries. - मुझे किराने का सामा...   
3    I love watching movies. - मुझे फिल्में देखना ब...   
4    Where is the nearest bank? - सबसे नज़दीकी बैंक...   
..                                                 ...   
295         Please write it down - कृपया इसे लिख दीजिए   
296               I need an umbrella - मुझे छाता चाहिए   
297               This is my friend - यह मेरा दोस्त है   
298    Do you have change? - क्या आपके पास छुट्टे हैं?   
299                 Let's take a break - चलो ब्रेक लें   

                                   English  \
0                 I am learning to drive.    
1    Can you recommend a good restaurant?    
2                I need to buy groceries.    
3                 I love watching movies.    
4              Where is the nearest bank?    
..                         

In [28]:
import sentencepiece as spm

# Train SentencePiece model
spm.SentencePieceTrainer.train(
    input='cleaned_sentences.csv',  # Use your dataset
    model_prefix='spm_model',
    vocab_size=3500,  # Adjusted vocabulary size
    character_coverage=0.9995,  # Covers Hindi, English, and special characters
    model_type='bpe'  # Byte-Pair Encoding
)


In [29]:
sp = spm.SentencePieceProcessor()
sp.load('spm_model.model')


True

In [30]:
import pandas as pd
import sentencepiece as spm

# Load SentencePiece model (train your own if needed)
sp = spm.SentencePieceProcessor()
sp.load('spm_model.model')  # Replace with your trained model path

# Load CSV file
file_path = 'cleaned_sentences.csv'  # Replace with your CSV file path
df = pd.read_csv(file_path)

# Tokenization using SentencePiece
df['English_Tokens'] = df['English'].apply(lambda x: sp.encode_as_pieces(str(x)))
df['Hindi_Tokens'] = df['Hindi'].apply(lambda x: sp.encode_as_pieces(str(x)))

# Save the tokenized data to a new CSV
output_path = 'tokenized_file.csv'
df.to_csv(output_path, index=False)

print(f"Tokenization complete! Data saved to {output_path}")


Tokenization complete! Data saved to tokenized_file.csv


In [31]:
print(f"Vocabulary size: {sp.get_piece_size()}")


Vocabulary size: 3500


In [33]:
import pandas as pd
import sentencepiece as spm
from sklearn.model_selection import train_test_split

# Load SentencePiece model (train your own if needed)
sp = spm.SentencePieceProcessor()
sp.load('spm_model.model')  # Replace with your trained model path

# Load CSV file
file_path = 'cleaned_sentences.csv'  # Replace with your CSV file path
df = pd.read_csv(file_path)

# Tokenization using SentencePiece
df['English_Tokens'] = df['English'].fillna('').apply(lambda x: sp.encode_as_pieces(str(x)))
df['Hindi_Tokens'] = df['Hindi'].fillna('').apply(lambda x: sp.encode_as_pieces(str(x)))

# Sentence alignment verification
df['Alignment_Flag'] = df.apply(lambda row: abs(len(row['English_Tokens']) - len(row['Hindi_Tokens'])) > 5, axis=1)
df_mismatch = df[df['Alignment_Flag']]
df_mismatch.to_csv('alignment_issues.csv', index=False)

# Split data into training (70%), validation (15%), and test (15%)
train_data, temp_data = train_test_split(df[~df['Alignment_Flag']], test_size=0.3, random_state=42)
val_data, test_data = train_test_split(temp_data, test_size=0.5, random_state=42)

# Save split data to new CSV files
train_data.to_csv('train_data.csv', index=False)
val_data.to_csv('val_data.csv', index=False)
test_data.to_csv('test_data.csv', index=False)

print("Data splitting complete! Files saved as 'train_data.csv', 'val_data.csv', and 'test_data.csv'.")
print("Alignment issues saved as 'alignment_issues.csv' for manual inspection.")


Data splitting complete! Files saved as 'train_data.csv', 'val_data.csv', and 'test_data.csv'.
Alignment issues saved as 'alignment_issues.csv' for manual inspection.


In [21]:
pip install indic-nlp-library


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.3/40.3 kB 1.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.7/7.7 MB 33.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.1/121.1 kB 11.4 MB/s eta 0:00:00


In [34]:
import nltk
nltk.download('wordnet')


[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [26]:
!pip install deep_translator


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.3/42.3 kB 1.9 MB/s eta 0:00:00


In [35]:
import random
import nltk
from nltk.corpus import wordnet
from deep_translator import GoogleTranslator
import re

# Download WordNet if not available
nltk.download('wordnet')

# Improved English augmentation logic
def augment_english(text):
    words = text.split()
    augmented_text = []

    for word in words:
        synonyms = wordnet.synsets(word)
        if synonyms:
            synonym_words = [lem.name().replace('_', ' ') for syn in synonyms for lem in syn.lemmas()]
            valid_synonyms = [w for w in synonym_words if re.match(r'^[a-zA-Z]+$', w)]
            if valid_synonyms:
                replacement = random.choice(valid_synonyms)
                augmented_text.append(replacement)
            else:
                augmented_text.append(word)
        else:
            augmented_text.append(word)

    return ' '.join(augmented_text)

# Hindi augmentation with transliteration
def augment_hindi(text):
    return GoogleTranslator(source='hi', target='en').translate(text)

# Punctuation cleanup
def clean_punctuation(text):
    return re.sub(r'(?<!\w)([.,!?])', r' \1', text).replace(' .', '.').replace(' ,', ',').replace(' ?', '?').replace(' !', '!')

# Sample Data
english_sentences = [
    "I want to visit a temple.",
    "I am interested in photography.",
    "Can I borrow your phone charger?",
]

hindi_sentences = [
    "मुझे एक मंदिर जाना है।",
    "मुझे फोटोग्राफी में रुचि है।",
    "क्या मैं आपका फोन चार्जर उधार ले सकता हूँ?",
]

# Process English and Hindi Augmentations
for eng, hin in zip(english_sentences, hindi_sentences):
    print(f"Original English: {eng}")
    print(f"Augmented English: {clean_punctuation(augment_english(eng))}")
    print(f"Original Hindi: {hin}")
    print(f"Augmented Hindi: {clean_punctuation(augment_hindi(hin))}")
    print('-' * 50)


[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


Original English: I want to visit a temple.
Augmented English: I require to visit A temple.
Original Hindi: मुझे एक मंदिर जाना है।
Augmented Hindi: I have to go to a temple.
--------------------------------------------------
Original English: I am interested in photography.
Augmented English: i personify interested in photography.
Original Hindi: मुझे फोटोग्राफी में रुचि है।
Augmented Hindi: I am interested in photography.
--------------------------------------------------
Original English: Can I borrow your phone charger?
Augmented English: can i borrow your headphone charger?
Original Hindi: क्या मैं आपका फोन चार्जर उधार ले सकता हूँ?
Augmented Hindi: Can I borrow your phone charger?
--------------------------------------------------


In [36]:
import sentencepiece as spm
import pandas as pd
import numpy as np

# Load SentencePiece model
sp = spm.SentencePieceProcessor()
sp.load("spm_model.model")

# Function to tokenize and convert sentences to token IDs
def tokenize_text(text, pad_id=0, max_length=50):
    token_ids = sp.encode(text, out_type=int)
    padded_token_ids = token_ids[:max_length] + [pad_id] * max(0, max_length - len(token_ids))
    mask = [1] * len(token_ids[:max_length]) + [0] * max(0, max_length - len(token_ids))
    return padded_token_ids, mask

# Process data files
def process_file(file_path, output_file):
    data = pd.read_csv(file_path)

    # Assume columns: 'English' and 'Hindi' exist
    data[['English_Token_Ids', 'English_Mask']] = data['English'].apply(lambda x: pd.Series(tokenize_text(str(x))))
    data[['Hindi_Token_Ids', 'Hindi_Mask']] = data['Hindi'].apply(lambda x: pd.Series(tokenize_text(str(x))))

    # Save the processed data
    data.to_csv(output_file, index=False)
    print(f"Processed data saved to {output_file}")

# Process train, validation, and test data
process_file('train_data.csv', 'train_tokenized.csv')
process_file('val_data.csv', 'val_tokenized.csv')
process_file('test_data.csv', 'test_tokenized.csv')


Processed data saved to train_tokenized.csv
Processed data saved to val_tokenized.csv
Processed data saved to test_tokenized.csv


In [31]:
import pandas as pd

# Load CSV
df = pd.read_csv('train_tokenized.csv')

# Display first few rows
print(df.head())


                                                   0  \
0  Do you speak English? - क्या आप अंग्रेज़ी बोलत...   
1  I want to visit a temple. - मुझे एक मंदिर जाना...   
2                         Let's go now - चलो अब चलें   
3  I am interested in photography. - मुझे फोटोग्र...   
4  Can I borrow your phone charger? - क्या मैं आप...   

                             English  \
0             Do you speak English?    
1         I want to visit a temple.    
2                      Let's go now    
3   I am interested in photography.    
4  Can I borrow your phone charger?    

                                         Hindi  \
0                 क्या आप अंग्रेज़ी बोलते हैं?   
1                       मुझे एक मंदिर जाना है।   
2                                  चलो अब चलें   
3                 मुझे फोटोग्राफी में रुचि है।   
4   क्या मैं आपका फोन चार्जर उधार ले सकता हूँ?   

                                      English_Tokens  \
0         ['▁Do', '▁you', '▁speak', '▁English', '?']   
1  ['▁I',

In [32]:
import pandas as pd

# Load CSV
df = pd.read_csv('train_tokenized.csv')

# Display the column names
print(df.columns)


Index(['0', 'English', 'Hindi', 'English_Tokens', 'Hindi_Tokens',
       'Alignment_Flag', 'English_Token_Ids', 'English_Mask',
       'Hindi_Token_Ids', 'Hindi_Mask'],
      dtype='object')


In [33]:
# For English tokens
sequence_lengths = df['English_Token_Ids'].apply(lambda x: len(eval(x)))

# For Hindi tokens
sequence_lengths = df['Hindi_Token_Ids'].apply(lambda x: len(eval(x)))

print(sequence_lengths.describe())



count    209.0
mean      50.0
std        0.0
min       50.0
25%       50.0
50%       50.0
75%       50.0
max       50.0
Name: Hindi_Token_Ids, dtype: float64


In [34]:
print(df[['English_Token_Ids', 'Hindi_Token_Ids']].head(5))


                                   English_Token_Ids  \
0  [357, 47, 461, 956, 3415, 0, 0, 0, 0, 0, 0, 0,...   
1  [12, 209, 26, 462, 11, 1672, 3444, 0, 0, 0, 0,...   
2  [558, 3467, 3410, 140, 320, 0, 0, 0, 0, 0, 0, ...   
3  [12, 58, 981, 245, 1787, 3444, 0, 0, 0, 0, 0, ...   
4  [150, 12, 937, 171, 289, 958, 3415, 0, 0, 0, 0...   

                                     Hindi_Token_Ids  
0  [39, 51, 978, 1626, 67, 3415, 0, 0, 0, 0, 0, 0...  
1  [16, 108, 1629, 287, 6, 3445, 0, 0, 0, 0, 0, 0...  
2  [321, 362, 607, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...  
3  [16, 1782, 208, 891, 6, 3445, 0, 0, 0, 0, 0, 0...  
4  [39, 25, 210, 322, 950, 605, 431, 105, 36, 341...  


In [46]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class TransformerTranslator(nn.Module):
    def __init__(self, input_dim, output_dim, emb_dim=256, n_heads=8, n_layers=4, dropout=0.1):
        super(TransformerTranslator, self).__init__()

        self.encoder = nn.TransformerEncoder(
            nn.TransformerEncoderLayer(d_model=emb_dim, nhead=n_heads, dropout=dropout),
            num_layers=n_layers
        )

        self.decoder = nn.TransformerDecoder(
            nn.TransformerDecoderLayer(d_model=emb_dim, nhead=n_heads, dropout=dropout),
            num_layers=n_layers
        )

        self.src_embedding = nn.Embedding(input_dim, emb_dim)
        self.tgt_embedding = nn.Embedding(output_dim, emb_dim)
        self.output_layer = nn.Linear(emb_dim, output_dim)

    def forward(self, src, tgt, src_mask=None, tgt_mask=None):
        src_emb = self.src_embedding(src)
        tgt_emb = self.tgt_embedding(tgt)

        encoder_output = self.encoder(src_emb, src_mask)
        decoder_output = self.decoder(tgt_emb, encoder_output, tgt_mask)

        return self.output_layer(decoder_output)


In [47]:
class PositionalEncoding(nn.Module):
    def __init__(self, emb_dim, max_len=1000):
        super(PositionalEncoding, self).__init__()
        pe = torch.zeros(max_len, emb_dim)
        position = torch.arange(0, max_len).unsqueeze(1)
        div_term = torch.exp(torch.arange(0, emb_dim, 2) * -(torch.log(torch.tensor(10000.0)) / emb_dim))

        pe[:, 0::2] = torch.sin(position * div_term)
        pe[:, 1::2] = torch.cos(position * div_term)

        self.pe = pe.unsqueeze(0)

    def forward(self, x):
        return x + self.pe[:, :x.size(1)]


In [48]:
from collections import Counter

def build_vocab(column):
    vocab_counter = Counter(token for tokens in column for token in tokens)
    vocab = {word: idx for idx, (word, _) in enumerate(vocab_counter.most_common())}

    # Add special tokens
    vocab['<PAD>'] = len(vocab)
    vocab['<SOS>'] = len(vocab)
    vocab['<EOS>'] = len(vocab)

    return vocab


In [49]:
english_vocab = build_vocab(df['English_Tokens'])
hindi_vocab = build_vocab(df['Hindi_Tokens'])

input_dim = len(english_vocab)  # Size of English vocabulary
output_dim = len(hindi_vocab)   # Size of Hindi vocabulary


In [50]:
import torch

def encode_sentences(column, vocab):
    return [[vocab.get(token, vocab['<PAD>']) for token in tokens] for tokens in column]

df['English_Encoded'] = encode_sentences(df['English_Tokens'], english_vocab)
df['Hindi_Encoded'] = encode_sentences(df['Hindi_Tokens'], hindi_vocab)


In [51]:
input_dim = len(english_vocab)  # Size of English vocabulary
output_dim = len(hindi_vocab)    # Size of Hindi vocabulary

model = TransformerTranslator(input_dim, output_dim)


In [52]:
pad_token_id = 0

criterion = nn.CrossEntropyLoss(ignore_index=pad_token_id)
optimizer = torch.optim.Adam(model.parameters(), lr=0.0005)


In [53]:
from transformers import AutoTokenizer

# Initialize tokenizer (e.g., for BERT or a custom model)
tokenizer = AutoTokenizer.from_pretrained('bert-base-uncased')  # or another model of your choice

# Set padding token (if not already defined)
tokenizer.pad_token = '[PAD]'

# Now you can access the padding token ID
pad_token_id = tokenizer.pad_token_id
print(f"Padding Token ID: {pad_token_id}")


Padding Token ID: 0


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [54]:
print(f"Padding Token ID: {tokenizer.pad_token_id}")


Padding Token ID: 0


In [55]:
criterion = nn.CrossEntropyLoss(ignore_index=pad_token_id)
optimizer = torch.optim.Adam(model.parameters(), lr=0.0005)


In [56]:
def train(model, data_loader, criterion, optimizer, device):
    model.train()
    total_loss = 0

    for src, tgt in data_loader:
        src, tgt = src.to(device), tgt.to(device)

        optimizer.zero_grad()
        output = model(src, tgt[:, :-1])

        loss = criterion(output.view(-1, output_dim), tgt[:, 1:].reshape(-1))
        loss.backward()
        optimizer.step()

        total_loss += loss.item()

    return total_loss / len(data_loader)


In [46]:
!pip install torchtext


In [57]:
from torchtext.data.metrics import bleu_score

def evaluate_bleu(model, data_loader, device):
    model.eval()
    candidates, references = [], []

    with torch.no_grad():
        for src, tgt in data_loader:
            src, tgt = src.to(device), tgt.to(device)
            output = model(src, tgt[:, :-1]).argmax(dim=-1)
            candidates.append(output.tolist())
            references.append(tgt.tolist())

    return bleu_score(candidates, references)


In [48]:
pip uninstall torch torchtext -y


Found existing installation: torch 2.0.0
Uninstalling torch-2.0.0:
  Successfully uninstalled torch-2.0.0
Found existing installation: torchtext 0.15.1
Uninstalling torchtext-0.15.1:
  Successfully uninstalled torchtext-0.15.1


In [3]:
pip install torchtext==0.15.1


In [58]:
import torch
import torchtext
print(torch.__version__)
print(torchtext.__version__)



2.0.0+cu117
0.15.1+cpu


In [59]:
from torchtext.data.metrics import bleu_score

def evaluate_bleu(model, data_loader, device):
    model.eval()
    candidates, references = [], []

    with torch.no_grad():
        for src, tgt in data_loader:
            src, tgt = src.to(device), tgt.to(device)
            output = model(src, tgt[:, :-1]).argmax(dim=-1)
            candidates.append(output.tolist())
            references.append(tgt.tolist())

    return bleu_score(candidates, references)


In [60]:
import torch.optim as optim

optimizer = optim.Adam(model.parameters(), lr=0.001)


In [61]:
import torch.nn.functional as F
from torch.utils.data import DataLoader, TensorDataset

# Example Loss Function
criterion = nn.CrossEntropyLoss()

# Example Data Loader
train_loader = DataLoader(TensorDataset(torch.randn(100, 784), torch.randint(0, 10, (100,))), batch_size=32)


In [62]:
from torch.optim import lr_scheduler

scheduler = lr_scheduler.StepLR(optimizer, step_size=5, gamma=0.5)


In [89]:
def train(model, train_loader, optimizer, criterion, vocab_size):
    model.train()
    total_loss = 0.0

    for batch in train_loader:
        src, tgt = batch

        # Debugging step to inspect tensor shapes
        print(f"Batch shapes -> src: {src.shape}, tgt: {tgt.shape}")

        # Ensure tensors are at least 2D for shape consistency
        if src.dim() == 1:
            src = src.unsqueeze(0)  # Convert [seq_len] to [1, seq_len]
        if tgt.dim() == 1:
            tgt = tgt.unsqueeze(0)

        # Handle empty sequences to avoid shape issues
        if src.size(1) == 0 or tgt.size(1) == 0:
            print("Empty batch detected. Skipping...")
            continue

        # Ensure tensors are in correct format and within valid range
        src = src.clamp(0, vocab_size - 1).long()
        tgt = tgt.clamp(0, vocab_size - 1).long()

        optimizer.zero_grad()

        # Model expects (src, tgt[:, :-1]) as input and (tgt[:, 1:]) as label
        outputs = model(src, tgt[:, :-1])
        loss = criterion(outputs, tgt[:, 1:])

        loss.backward()
        optimizer.step()

        total_loss += loss.item()

    avg_loss = total_loss / len(train_loader)
    return avg_loss


In [90]:
import math

def calculate_perplexity(loss):
    return math.exp(loss)


In [91]:
torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)


tensor(0.)

In [92]:
for batch in train_loader:
    src, tgt = batch


In [93]:
for batch in train_loader:
    print(batch)  # Inspect what `batch` contains
    break

[tensor([[-0.7358, -0.9415,  1.0479,  ..., -0.0088, -3.1458,  0.8570],
        [-0.2672,  2.2041,  1.1032,  ..., -0.7124, -1.2057, -0.3686],
        [-0.9392,  0.2753, -0.5374,  ..., -0.2834, -3.3172,  1.3903],
        ...,
        [ 0.1872,  1.4039, -0.1594,  ...,  0.4510,  0.0265, -0.5584],
        [ 1.4610, -0.1598, -0.5088,  ...,  0.2585,  1.2573, -1.2443],
        [ 1.3106, -0.9712, -1.9561,  ...,  0.3724,  0.4874,  1.2768]]), tensor([8, 6, 8, 9, 3, 6, 7, 8, 3, 5, 9, 6, 6, 2, 6, 2, 0, 9, 4, 4, 9, 3, 6, 8,
        3, 8, 4, 2, 9, 4, 5, 9])]


In [94]:
vocab_size = 3000  # Or set it to your actual vocabulary size
epochs = 20  # Define number of epochs

for epoch in range(epochs):
    train_loss = train(model, train_loader, optimizer, criterion, vocab_size)

    # Gradient Clipping
    torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)

    # Learning Rate Scheduler
    scheduler.step(train_loss)

    print(f"Epoch {epoch+1} - Training Loss: {train_loss:.4f}")

# Save the trained model AFTER the training loop
torch.save(model.state_dict(), "translation_model.pt")
print("Model saved successfully!")


Batch shapes -> src: torch.Size([32, 784]), tgt: torch.Size([32])


RuntimeError: shape '[32, 248, 32]' is invalid for input of size 6422528

In [15]:
from nltk.translate.bleu_score import corpus_bleu

def calculate_bleu(references, hypotheses):
    score = corpus_bleu([[ref] for ref in references], hypotheses)
    return score

# Example usage
references = ["this is a test", "another sentence example"]
hypotheses = ["this is a test", "another sample sentence"]
print("BLEU Score:", calculate_bleu(references, hypotheses))


BLEU Score: 0.8740063190188735


In [16]:
!pip install rouge-score


In [17]:
from rouge_score import rouge_scorer

def calculate_rouge(references, hypotheses):
    scorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL'], use_stemmer=True)
    scores = [scorer.score(ref, hyp) for ref, hyp in zip(references, hypotheses)]
    return scores

# Example usage
references = ["this is a test", "another sentence example"]
hypotheses = ["this is a test", "another sample sentence"]
print("ROUGE Scores:", calculate_rouge(references, hypotheses))


ROUGE Scores: [{'rouge1': Score(precision=1.0, recall=1.0, fmeasure=1.0), 'rouge2': Score(precision=1.0, recall=1.0, fmeasure=1.0), 'rougeL': Score(precision=1.0, recall=1.0, fmeasure=1.0)}, {'rouge1': Score(precision=0.6666666666666666, recall=0.6666666666666666, fmeasure=0.6666666666666666), 'rouge2': Score(precision=0.0, recall=0.0, fmeasure=0.0), 'rougeL': Score(precision=0.6666666666666666, recall=0.6666666666666666, fmeasure=0.6666666666666666)}]


In [20]:
def human_evaluation(translations):
    scores = []

    print("Rate each translation on a scale of 1 to 5:")
    for idx, item in enumerate(translations):
        print(f"\nSample {idx + 1}")
        print(f"English Sentence: {item['english']}")
        print(f"Model Translation: {item['model_translation']}")
        print(f"Reference Translation: {item['reference_translation']}")

        fluency = int(input("Fluency (1-5): "))
        adequacy = int(input("Adequacy (1-5): "))
        overall_quality = int(input("Overall Quality (1-5): "))

        scores.append({
            'fluency': fluency,
            'adequacy': adequacy,
            'overall_quality': overall_quality
        })

    # Calculate averages
    avg_fluency = sum(score['fluency'] for score in scores) / len(scores)
    avg_adequacy = sum(score['adequacy'] for score in scores) / len(scores)
    avg_quality = sum(score['overall_quality'] for score in scores) / len(scores)

    print("\n--- Evaluation Results ---")
    print(f"Average Fluency: {avg_fluency:.2f}")
    print(f"Average Adequacy: {avg_adequacy:.2f}")
    print(f"Average Overall Quality: {avg_quality:.2f}")

# Sample Data
evaluation_data = [
    {"english": "The weather is nice today.",
     "model_translation": "आज का मौसम अच्छा है।",
     "reference_translation": "आज मौसम अच्छा है।"},
    {"english": "I love programming.",
     "model_translation": "मुझे प्रोग्रामिंग पसंद है।",
     "reference_translation": "मैं प्रोग्रामिंग पसंद करता हूँ।"}
]

human_evaluation(evaluation_data)


Rate each translation on a scale of 1 to 5:

Sample 1
English Sentence: The weather is nice today.
Model Translation: आज का मौसम अच्छा है।
Reference Translation: आज मौसम अच्छा है।
Fluency (1-5): 3
Adequacy (1-5): 4
Overall Quality (1-5): 5

Sample 2
English Sentence: I love programming.
Model Translation: मुझे प्रोग्रामिंग पसंद है।
Reference Translation: मैं प्रोग्रामिंग पसंद करता हूँ।
Fluency (1-5): 2
Adequacy (1-5): 3
Overall Quality (1-5): 4

--- Evaluation Results ---
Average Fluency: 2.50
Average Adequacy: 3.50
Average Overall Quality: 4.50


In [37]:
sp = spm.SentencePieceProcessor()
sp.Load("spm_model.model")


True

In [38]:
import torch
import sentencepiece as spm

# Load SentencePiece tokenizer
sp = spm.SentencePieceProcessor()
sp.load("spm_model.model")

# Load your trained model
class TranslationModel(torch.nn.Module):
    def __init__(self, vocab_size, embed_size, hidden_size):
        super(TranslationModel, self).__init__()
        self.embedding = torch.nn.Embedding(vocab_size, embed_size)
        self.lstm = torch.nn.LSTM(embed_size, hidden_size, batch_first=True)
        self.fc = torch.nn.Linear(hidden_size, vocab_size)

    def forward(self, x):
        embedded = self.embedding(x)
        output, _ = self.lstm(embedded)
        return self.fc(output)

vocab_size = len(sp)
model = TranslationModel(vocab_size, 256, 512)
model.load_state_dict(torch.load("translation_model.pt"), strict=False)
model.eval()

# Translation function
def translate(sentence):
    tokens = sp.encode(sentence, out_type=int)
    input_tensor = torch.tensor([tokens]).long()

    with torch.no_grad():
        output = model(input_tensor)

    translated_tokens = torch.argmax(output, dim=-1).squeeze().tolist()
    translated_sentence = sp.decode(translated_tokens)
    return translated_sentence

# Test the translation
sentence = "The sun rises in the east."
print("Translated Sentence:", translate(sentence))


RuntimeError: Error(s) in loading state_dict for TranslationModel:
	Missing key(s) in state_dict: "embedding.weight", "lstm.weight_ih_l0", "lstm.weight_hh_l0", "lstm.bias_ih_l0", "lstm.bias_hh_l0", "fc.weight", "fc.bias". 
	Unexpected key(s) in state_dict: "fc1.weight", "fc1.bias", "fc2.weight", "fc2.bias". 

In [27]:
import os
print(os.listdir())


['.config', 'train_data.csv', 'tokenized_file.csv', 'train_tokenized.csv', 'augmented_data.csv', '.ipynb_checkpoints', 'spm_model.model', 'cleaned_sentences.csv', 'test_data.csv', 'val_data.csv', 'eng-hin.csv', 'test_tokenized.csv', 'alignment_issues.csv', 'val_tokenized.csv', 'spm_model.vocab', 'sample_data']


In [31]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import pandas as pd
import sentencepiece as spm
import ast

# Define Dataset
class TranslationDataset(Dataset):
    def __init__(self, file_path):
        self.data = pd.read_csv(file_path)

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        src = torch.tensor(ast.literal_eval(self.data.iloc[idx, 0]))
        tgt = torch.tensor(ast.literal_eval(self.data.iloc[idx, 1]))
        return src, tgt

# Translation Model
class TranslationModel(nn.Module):
    def __init__(self, vocab_size, embed_dim, hidden_dim):
        super(TranslationModel, self).__init__()
        self.embedding = nn.Embedding(vocab_size, embed_dim)
        self.encoder = nn.LSTM(embed_dim, hidden_dim, batch_first=True)
        self.decoder = nn.Linear(hidden_dim, vocab_size)

    def forward(self, x):
        embedded = self.embedding(x)
        _, (hidden, _) = self.encoder(embedded)
        output = self.decoder(hidden.squeeze(0))
        return output

# Load SentencePiece model
sp = spm.SentencePieceProcessor()
sp.Load("spm_model.model")

# Load Model
vocab_size = len(sp)
model = TranslationModel(vocab_size, 256, 512)

try:
    model.load_state_dict(torch.load("translation_model.pt"))
    print("Model loaded successfully!")
except FileNotFoundError:
    print("Error: Model file not found. Please train and save the model first.")

model.eval()

def translate_sentence(sentence):
    tokens = sp.encode(sentence, out_type=str)
    token_ids = torch.tensor(sp.encode(sentence)).unsqueeze(0)  # Add batch dim

    with torch.no_grad():
        output = model(token_ids)
        predicted_ids = torch.argmax(output, dim=-1).squeeze().tolist()

    translated_sentence = sp.decode(predicted_ids)
    return translated_sentence

# Testing the model with sample sentence
test_sentence = "The sun rises in the east."
print(f"English: {test_sentence}")
print(f"Hindi Translation: {translate_sentence(test_sentence)}")


Error: Model file not found. Please train and save the model first.
English: The sun rises in the east.
Hindi Translation: बिल
